In [2]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os import path as op
import itertools
import re
import xarray as xr
from sklearn.model_selection import train_test_split
import numpy.random as nr

%matplotlib inline

/scratch/adomakor412/conda/envs/MyEnv/lib/python3.6/site-packages/dask/config.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
nr.seed = 42

In [4]:
inputPath = '/scratch/adomakor412/all_npy3'
PATH = os.path.expanduser(inputPath)
ncPath = os.path.expanduser('/scratch/adomakor412/april_data_2019')

In [5]:
columns = [
    "time",
    "longitude",
    "latitude",
    #"band",#Necessary?
    "G17_Temp",
    "G17_mean",
    "G17_std",#mean and std outside inner for loop for comp. eff.
    "target_G16_Temp"
]

df = pd.DataFrame({},columns = columns)

In [6]:
def longitude(lonMin, lonMax, col, colSize):
    lon = (col/colSize)* (lonMax - lonMin)
    return lon

def latitude(latMin, latMax, row, rowSize):
    lat = (row/rowSize)* (latMax - latMin)
    return lat

def Rad2BT(rad, planck_fk1, planck_fk2, planck_bc1, planck_bc2):
    """Radiances to Brightness Temprature (using black body equation)"""
    invRad = np.array(rad)**(-1)
    arg = (invRad*planck_fk1) + 1.0
    T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2) 
    return T

In [7]:
import logging
logger = logging.getLogger()

In [126]:
extent_pc = [-109.59326, -102.40674, 8.94659, -8.94656]

dataFrames = []
inds=nr.randint(10000, size=(1000))
inds.sort()

with open('logML', 'a') as log:
    
    for (bb,dd) in list(itertools.product([8],[5])):#Let's start with one day
        DD = str(98+dd).zfill(3)
        lookup = f'M6C08_G17_s2019{DD}0000'
        ncFiles = [f for f in os.listdir(ncPath) if re.search(lookup,f)]
        #ncFiles = mySort(ncFiles)
        npFiles = [f for f in os.listdir( PATH ) if re.search(lookup,f)]
        #ncFiles = mySort(ncFiles)

        for ncf, npf in zip(ncFiles,npFiles):
            try:
                imageBox = np.load(op.join( PATH,npf))
                myFile = xr.open_dataset(op.join(ncPath,ncf))
                planck_fk1 = float(myFile['planck_fk1'].data)
                planck_fk2 = float(myFile['planck_fk2'].data) 
                planck_bc1 = float(myFile['planck_bc1'].data)                       
                planck_bc2 = float(myFile['planck_bc2'].data)

                time = ncf[31:38]
                G17_mean = Rad2BT(imageBox.mean(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)
                G17_std = Rad2BT(imageBox.std(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)

                hh = ncf[34:36]
                mm = ncf[36:38]
                
                print(ncf)
                print(str(ncf), file=log)
                logger.info(str(ncf))

                G16_npy = np.load( op.join(PATH, npf.replace('G16','G17',1)) )
                G16_ncf = xr.open_dataset(op.join( ncPath, ncf.replace('G16','G17',1) ))
                G16_fk1 = float(G16_ncf['planck_fk1'].data)
                G16_fk2 = float(G16_ncf['planck_fk2'].data) 
                G16_bc1 = float(G16_ncf['planck_bc1'].data)                       
                G16_bc2 = float(G16_ncf['planck_bc2'].data)

                target_G16_Temp = Rad2BT(G16_npy, G16_fk1, G16_fk2, G16_bc1, G16_bc2)
                x,y = imageBox.shape[0],imageBox.shape[1]
                randInds = nr.randint(10000, size=(x*y))
                
                #select 10,000 random points out of 401,401
                sample = np.array([combo for combo in itertools.product(range(x),range(y))])
                for i,j in sample[inds]:
                    lon = longitude( extent_pc[0], extent_pc[1], i, x )
                    lat = latitude( extent_pc[2], extent_pc[3], j, y)

                    G17_Temp = Rad2BT(imageBox[i,j], planck_fk1, planck_fk2, planck_bc1, planck_bc2)#unfiltered
                    target_G16_Temp = Rad2BT( G16_npy[i,j], G16_fk1, G16_fk2, G16_bc1, G16_bc2 )
                    
                    row = [time, lon, lat, G17_Temp, G17_mean, G17_std, target_G16_Temp]

                    dataFrames.append(pd.DataFrame([row],columns=columns))
                    #concatenate a list of single dataframes after done with loop for speed " pandas.concat"
                    #z score versus the standard deviation for single image, can normalize across all image
                
            except ValueError as e:
                logger.exception(e)
                print(e)
                print(e, file=log)
                continue
                 
    
# df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", names=columns)
# for column in df.columns:
#     if df[column].dtype == "object":
#         df[column] = df[column].str.strip()
# df = df.dropna()

OR_ABI-L1b-RadF-M6C08_G17_s20191030000339_e20191030009405_c20191030009441.nc


In [127]:
df=pd.concat(dataFrames)

In [128]:
from sklearn.neural_network import MLPRegressor
#from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [129]:
print(len(df))
print(df.head())

1000
      time  longitude  latitude    G17_Temp    G17_mean     G17_std  \
0  1030000        0.0 -1.070912  242.713931  243.715676  202.188943   
0  1030000        0.0 -1.294018  242.919616  243.715676  202.188943   
0  1030000        0.0 -2.231066  242.506926  243.715676  202.188943   
0  1030000        0.0 -2.275687  242.610594  243.715676  202.188943   
0  1030000        0.0 -2.320309  242.506926  243.715676  202.188943   

   target_G16_Temp  
0       242.713931  
0       242.919616  
0       242.506926  
0       242.610594  
0       242.506926  


In [130]:
dfSample = df.dropna().sample(n=1000)
X = dfSample.drop(["target_G16_Temp"],axis=1).astype('float')#try as float
y = dfSample.drop(columns[:-1],axis=1).astype('float')

In [131]:
X.head()

,time,longitude,latitude,G17_Temp,G17_mean,G17_std
0,1030000.0,0.143587,-11.958514,244.373029,243.715676,202.188943
0,1030000.0,0.071793,-16.732996,244.859563,243.715676,202.188943
0,1030000.0,0.021538,-5.443801,243.174892,243.715676,202.188943
0,1030000.0,0.014359,-5.800772,243.123997,243.715676,202.188943
0,1030000.0,0.150766,-12.270863,244.373029,243.715676,202.188943


In [132]:
y.head()

,target_G16_Temp
0,244.373029
0,244.859563
0,243.174892
0,243.123997
0,244.373029


In [133]:
from sklearn.metrics import mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y),random_state=1,test_size=0.1)
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)

In [134]:
X_test[:5]

,time,longitude,latitude,G17_Temp,G17_mean,G17_std
0,1030000.0,0.014359,-1.472504,242.506926,243.715676,202.188943
0,1030000.0,0.043076,-14.680415,244.956011,243.715676,202.188943
0,1030000.0,0.007179,-1.963338,242.558805,243.715676,202.188943
0,1030000.0,0.050255,-10.396768,244.028100,243.715676,202.188943
0,1030000.0,0.086152,-11.958514,244.470914,243.715676,202.188943


In [135]:
regr.predict(X_test[:5])

array([243.08473969, 244.0658145 , 243.12858763, 243.76927625,
       243.8563336 ])

In [136]:
regr.score(X_test, y_test)

0.6073098655983802

In [137]:
prediction = regr.predict(X_test)

In [138]:
errors = abs(prediction - y_test)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'Kelvin')
print('Mean Square Error:', mean_squared_error(prediction, y_test))

Mean Absolute Error: 0.52 Kelvin
Mean Square Error: 0.36117510682413545


In [139]:
regr.score(X_train,y_train)

0.5976042935047217

### Previous MSE was 23.96 at 1000 points sampled. Going from 1000 to 10000 points acheived better MSE (23.96 -> 0.36), MAE (3.02 Kelvins -> 0.52 Kelvins) and R^2 of both the test (-0.42 -> 0.61) and train (-0.32 ->) data.

### Since the R^2 of the train and test data are similar, we know we are not overfitting.